In [1]:
%matplotlib inline

In [2]:
%run ~/src/starry/notebooks/notebook_setup.py

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import starry
from tqdm.notebook import tqdm
import theano
import theano.tensor as tt
from theano.tensor.shared_randomstreams import RandomStreams
from scipy.linalg import cho_factor, cho_solve

starry.config.lazy = False
starry.config.quiet = True

In [36]:
# Params
ydeg = 10
nspots = 10
npoints = 200
inc = 60
p = 0.237
sigma = 0.01
amp = -0.001
mu_lat1 = 0
std_lat1 = 2.5
mu_lat2 = 60
std_lat2 = 2.5

In [37]:
# Setup
np.random.seed(0)
map = starry.Map(ydeg, lazy=True)
t = np.linspace(0, 1, npoints)
_A = map.design_matrix(theta=360 / p * t)
A = _A.eval()
trand = RandomStreams(seed=234)

INFO (theano.gof.compilelock): Refreshing lock /Users/rluger/.theano/compiledir_Darwin-19.6.0-x86_64-i386-64bit-i386-3.7.4-64/lock_dir/lock


In [ ]:
# Compile our forward model
lon = trand.uniform([nspots]) * 360
lat1 = mu_lat1 + std_lat1 * trand.normal([nspots])
lat2 = mu_lat2 + std_lat2 * trand.normal([nspots])
lat = tt.switch(trand.uniform([nspots]) < 0.5, lat1, lat2)

def _draw_flux(alpha):
    y = [None for k in range(npoints)]
    for k, tk in tqdm(enumerate(t), total=npoints):
        map.reset(inc=inc)
        for i in range(nspots):
            map.add_spot(
                lon=lon[i] - 360.0 / p * alpha * tk * tt.sin(lat[i] * np.pi / 180) ** 2,
                lat=lat[i],
                amp=amp,
                sigma=sigma,
                relative=False,
            )
        y[k] = map.amp * map.y
    return tt.diag(tt.dot(_A, tt.as_tensor_variable(y).T))

_alpha = tt.dscalar()
draw_flux = theano.function([_alpha], _draw_flux(_alpha))

In [ ]:
# Compute empirical mean and covariance of the process
alpha = 0.25
nlc = 10000
flux = np.empty((nlc, npoints))

for k in tqdm(range(nlc)):
    flux[k] = draw_flux(alpha)
    
mu = np.mean(flux, axis=0)
L = np.cov(flux.T)
cho_L = cho_factor(L + 1e-12 * np.eye(npoints), lower=True)[0]

In [ ]:
# Draw light curves w/ different alpha
alpha = np.linspace(0.01, 0.5, 250)
ferr = 1e-4
flux = [draw_flux(a) + ferr * np.random.randn(npoints) for a in alpha]

In [ ]:
# Compute the likelihood for each given GP(alpha=0.25)
# Should peak at alpha = 0.25!
lnlike = [starry.linalg.lnlike(np.eye(npoints), flux[k], C=ferr ** 2, cho_L=cho_L, mu=mu) for k in range(250)]

In [ ]:
# Super noisy, but yes!
plt.plot(alpha, (lnlike - np.max(lnlike)));
plt.ylim(-150, 0);

In [ ]:
plt.imshow(L)

In [ ]:
k = L[:, 0]
plt.axhline(0, color="k", lw=1, alpha=0.5, ls="--")
for t in [0, 1, 2]:
    plt.axvline(t, color="k", lw=1, alpha=0.5, ls="--")
plt.plot(np.linspace(0, 1/p, 200), k / k.max());
plt.xlabel("periods")
plt.ylabel("covariance");